In [1]:
nb_tables = 22
nb_seances = 3

In [2]:
EXCEPTIONS = {2:[], 3:[], 4:[4], 5:[4], 6:[3], 
              7: [3, 5, 6, 7], 
              8: [5, 7, 8],
              9: [3, 4, 8],    
             10: [3, 4, 9],
             11: [3, 4, 9, 11],
             12: [6, 8, 11, 12],
             13: [4, 5, 7, 9, 10, 11, 12, 13],
             14: [3, 4, 5, 6, 7, 11, 12, 14],
             15: [5, 6, 7, 9, 10, 11, 13, 14],
             16: [4, 7, 12, 13, 14 ,16],
             17: [4, 9, 10, 13, 15, 16, 17],
             18: [3, 5, 8, 9, 11, 12, 13, 14, 15],
             19: [3, 4, 6, 8, 15, 16],
             20: [5, 9, 11, 14, 17, 18, 19],
             21: [3, 5, 8, 9, 11, 12, 18],
             22: [3, 4, 6, 8, 9, 13, 19, 20, 21],
             23: [3, 4, 5, 8, 11, 13, 14, 15, 17 ,18 ,19, 20],
             24: [6, 11, 12, 14, 16, 20 , 21, 23, 24],
             25: [4, 11, 12, 13, 18, 21, 22, 24],
             }                    

In [3]:
from math import ceil

In [4]:
def decouper_tour(nb_tours, nb_seances):
    """ Découpe les tours en les répartissant équitablement sur le nb de séances """
    if nb_seances == 1:
        return [nb_tours]
    else:
        nb_tour_seance_1 = ceil(nb_tours / nb_seances)
        return [nb_tour_seance_1] + decouper_tour(nb_tours-nb_tour_seance_1, nb_seances-1)
        
#decoupage = decouper_tour(2*nb_tables-1, nb_seances)
#decoupage

In [5]:
def positions_init(nb_tables):
    """ Renvoie une liste des positions initiales en fonction du nb de tables """
    exceptions = EXCEPTIONS[nb_tables]
    positions = {NumPaire:{'NumTable':None, 'NumAdversaire':None, 'Position':None} for NumPaire in range(1, 2*nb_tables+1)}
    positions = {NumPaire:{} for NumPaire in range(1, 2*nb_tables+1)}
    for NumPaire in range(1, nb_tables+1):
        NumAdversaire = 2*nb_tables+1 - NumPaire # la somme des n° de paires est : 2*nb_tables + 1
        positions[NumPaire]['NumTable'] = NumPaire
        positions[NumAdversaire]['NumTable'] = NumPaire 
        positions[NumPaire]['NumAdversaire'] = NumAdversaire
        positions[NumAdversaire]['NumAdversaire'] = NumPaire
        if NumPaire in exceptions:
            positions[NumPaire]['Position'] = 'NS'
            positions[NumAdversaire]['Position'] = 'EO' 
        else:
            positions[NumPaire]['Position'] = 'EO'
            positions[NumAdversaire]['Position'] = 'NS' 
    return positions

#test
#positions_init(nb_tables)

In [6]:
def mouvement(positions):
    """ Renvoie la nouvelle carte des positions après un changement de positions """
    nb_paires = len(positions)
    NumTable_dernier, Position_dernier = positions[nb_paires-1]['NumTable'], positions[nb_paires-1]['Position']
    ### maj des tables et positions : ###
    for NumPaire in range(nb_paires-1, 1, -1):
        positions[NumPaire]['NumTable'] = positions[NumPaire-1]['NumTable']
        positions[NumPaire]['Position'] = positions[NumPaire-1]['Position']
    positions[1]['NumTable'], positions[1]['Position'] = NumTable_dernier, Position_dernier
    ### maj des adversaires : ###
    for NumPaire1 in range(1, nb_paires+1):
        for NumPaire2 in range(1, nb_paires+1):
            if (NumPaire1 != NumPaire2) and (positions[NumPaire2]['NumTable'] == positions[NumPaire1]['NumTable']):
                positions[NumPaire1]['NumAdversaire'] = NumPaire2
    return positions

#test
#mouvement(positions_init(nb_tables))

In [7]:
decoupage = decouper_tour(2*nb_tables-1, nb_seances)
seances = []
for i in range(nb_seances):
    seances.extend([i+1]*decoupage[i])

with open(f"barometre_{nb_tables}_tables.csv", "w") as f:
    f.write("NumSeance;NumTour;NumPaire;NumAdversaire;NumTable;Position;NumDonne1;NumDonne2\n")
            
    positions = positions_init(nb_tables)
    for tour in range(1, 2*nb_tables):        # tour = n° de tour "global" (sans découpage par séance)
        s = seances[tour-1]                   # s = n° de séance
        t = tour - (sum(decoupage[:s-1]))     # t = n° de tour "effectif" au sein d'une séance
        d1, d2 = 2*t-1, 2*t                   # d1, d2 = n° des 2 donnes jouées
        for NumPaire in sorted(positions):
            f.write(f"{s};{t};{NumPaire};{positions[NumPaire]['NumAdversaire']};{positions[NumPaire]['NumTable']};{positions[NumPaire]['Position']};{d1};{d2}\n")
        positions = mouvement(positions)

In [ ]:
import basthon

In [ ]:
basthon.download(f"barometre_{nb_tables}_tables.csv")